## A demonstration of using Dask to visualize a density map of the full source catalog from the Gaia DR2 release

### Visualization imports

In [ ]:
import holoviews as hv
import holoviews.operation.datashader as hd

### Set up the cluster

Dask provides utilities to build clusters to use in distributed compute jobs. In this particular case we will use a `KubeCluster`.  This is a type of cluster that knows how to use the kubernetes API to spawn separate pods for each worker.  The description for each worker is in a special file in the `dask` directory of the home directory.

In [ ]:
from dask.distributed import Client, wait
from dask import dataframe as dd
from dask_kubernetes import KubeCluster
import os
cluster = KubeCluster.from_yaml(os.getenv('HOME') + '/dask/dask_worker.yml')

Since we are working in a dynamic environment where we don't know up front what ports/nodes our process will spawn on, there are a couple of utility classes that wrap dask classes to provide useful information about the running dask cluster.

In [ ]:
from jupyterlabutils.notebook import LSSTDaskClient, ClusterProxy

The workers have the same profile as the instance selected to run the notebook.  For this example, we suggest a `large` size with 4 cores and 12GB of RAM.  Scaling to 60 threads is a good number for this demo.

In [ ]:
import math, os
threads=60
cpuper = float(os.getenv('CPU_LIMIT') or '4.0')
workers = math.ceil(threads/cpuper)

_ = cluster.scale_up(workers)
client = LSSTDaskClient(address=cluster)
client

The link above will take you to the status dashboard for the summary information about the cluster.

The proxy in the next cell will give access to the status dashboards for each of the workers in your cluster.

In [ ]:
proxy = ClusterProxy(client)
proxy

Repeat the following until you've got a full complement, or nearly so, of workers.

In [ ]:
print(client)
print(proxy)

Now read the metadata for the parquet files we'll use for the analysis below.  This does not read all the data, but only the metadata for the files in this data set.

> Note that either of the methods in the cell will work when runing at the LDF, however direct posix filesystem access may not work if running in a different environment (e.g. GKE).

In [ ]:
df = dd.read_parquet('/project/shared/data/gaia_dr2/gaia_source.parquet', columns=['l', 'b'], engine='fastparquet')
#if reading from the cloud storage bucket, use the following instead
#df = dd.read_parquet('gcs://jupyterlabdemo-gaia-dr2/gaia_source.parquet', columns=['l', 'b'], engine='fastparquet')

We have asked for only a subset of the source catalog.  Specifically, only the galactic longitude and latitude.  We tell the dask client to cache these columns in memory on the worker nodes with the `persist` method to speed up computations in the future.
> Note that the `persist` method is asynchronous, so following cells that interact with the dataframe may not execute until the persist is finished.  Follow along with progress by visiting the link in the output of cell 4.

In [ ]:
df = df.persist()

We can now do things like count the number of rows.  This is still a parallel computation and you can look at the summary of the execution by going to the link in the output of cell 4.

In [ ]:
len(df)

We are going to produce an aggregate map over cells on the sky.  The default is to simply count up the number of entries in each spacial cell.  To set up the color map, we ask that the smallest numbers be shown in light blue and the largest in darkblue with a linear ramp.

In [ ]:
print(client._timeout)
print(client.overall_timeout)

In [ ]:
hd.shade.cmap=["lightblue", "darkblue"]
hv.extension("bokeh", "matplotlib")

Set up the points to be aggregated.  Defaults are fine here.

In [ ]:
points = hv.Points(df)

Now do the aggregation and display.  The `datashade` method will bin each of our two spacial coordinates and sum the entries in each.  This effectively produces a density map of the sky for all 1.7 billion entries in the Gaia DR2 source catalog.  Since we are using bokeh as the rendering library, the standard pan and zoom widgets are available.

In [ ]:
%%time
%%opts RGB [width=1000, height=500]
hd.datashade(points)

In [ ]:
# close down the cluster
cluster.close()